In [9]:
import sys
import re
import pymongo
import json
import time
import datetime
import requests
from bs4 import BeautifulSoup

dbname = "fdac18mp2" #please use this database
collname = "glprj_cflemmon" #please modify so you store data in your collection
myLetter = 'z'

# beginning page index
begin = "1"
client = pymongo.MongoClient()

db = client[dbname]
coll = db[collname]

beginurl = "https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=" + begin + \
    "&per_page=99&simple=false&sort=desc&starred=false&statistics=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false"

gleft = 20

header = {'per_page': 99}

#source_url = "https://sourceforge.net/directory/?q=" + myLetter + "&sort=name&page="
#rest_url = "https://sourceforge.net/rest/p/"

# check remaining query chances for rate-limit restriction
def wait(left):
    global header
    while (left < 20):
        l = requests.get('https://gitlab.com/api/v4/projects', headers=header)
        if (l.ok):
            left = int(l.headers.get('RateLimit-Remaining'))
        time .sleep(60)
    return left

def project_exists(url):
    r = requests.get(url)
    if r.status_code == 200:
        return True
    return False

'''
def get_source(url, coll, rest):
    page = 1
    project_count = 0
    while True:
        resp = requests.get(url + str(page))
        text = resp.text
        soup = BeautifulSoup(text, 'html.parser')
        if re.search('No results found.', soup.get_text()):
            return
        
        for link in soup.find_all(class_="project-icon", href=True):
            name = re.findall('/projects/([A-Za-z0-9\-]*)', link.get('href'))
            name = name[0] if name else None
            if name is not None and name.lower().startswith(myLetter):
                resp = requests.get(rest + name)
                if resp.status_code == 200:
                    info = json.loads(resp.text)
                    info['forge'] = 'sourceforge'
                    coll.insert_one(info)
                    project_count += 1
                    if project_count >= 50:
                        return
        page += 1
    return
'''

def get_gitlab(url, coll):
    
    global gleft
    global header
    global bginnum
    gleft = wait(gleft)
    values = []
    size = 0
    project_count = 0

    try:
        r = requests .get(url, headers=header)
        time .sleep(0.5)
        # got blocked
        if r.status_code == 403:
            return "got blocked", str(bginnum)
        if (r.ok):

            gleft = int(r.headers.get('RateLimit-Remaining'))
            lll = r.headers.get('Link')
            t = r.text
            array = json.loads(t)
            
            for el in array:
                if el['name'].lower().startswith(myLetter):
                    if project_exists(el['http_url_to_repo']):
                        project_count += 1
                        el['forge'] = 'gitlab'
                        coll.insert_one(el)
                        if project_count >= 50:
                            return
 
            #next page
            while ('; rel="next"' in lll):
                gleft = int(r.headers.get('RateLimit-Remaining'))
                gleft = wait(gleft)
                # extract next page url
                ll = lll.replace(';', ',').split(',')
                url = ll[ll.index(' rel="next"') -
                         1].replace('<', '').replace('>', '').lstrip()
             
                try:
                    r = requests .get(url, headers=header)
                    if r.status_code == 403:
                        return "got blocked", str(bginnum)
                    if (r.ok):
                        lll = r.headers.get('Link')
                        t = r.text
                        array1 = json.loads(t)
                        for el in array1:
                            if el['name'].lower().startswith(myLetter):
                                if project_exists(el['http_url_to_repo']):
                                    project_count += 1
                                    el['forge']
                    else:
                        sys.stderr.write("url can not found:\n" + url + '\n')
                        return 
                except requests.exceptions.ConnectionError:
                    sys.stderr.write('could not get ' + url + '\n')

        else:
            sys.stderr.write("url can not found:\n" + url + '\n')
            return

    except requests.exceptions.ConnectionError:
        sys.stderr.write('could not get ' + url + '\n')
    except Exception as e:
        sys.stderr.write(url + ';' + str(e) + '\n')
        
#start retrieving 
get_gitlab(beginurl, coll)
#get_source(source_url, coll, rest_url)


https://gitlab.com/api/v4/projects?archived=false&membership=false&order_by=created_at&owned=false&page=2&per_page=99&repository_checksum_failed=false&simple=false&sort=desc&starred=false&statistics=false&wiki_checksum_failed=false&with_custom_attributes=false&with_issues_enabled=false&with_merge_requests_enabled=false;'forge'


In [10]:
#print collected data
for doc in coll.find({}):
    print(doc)

{'web_url': 'https://gitlab.com/stupid-conference/stream-hud', 'path': 'stream-hud', 'name': 'stream-hud', 'ssh_url_to_repo': 'git@gitlab.com:stupid-conference/stream-hud.git', 'namespace': {'id': 3840493, 'path': 'stupid-conference', 'name': 'stupid-conference', 'kind': 'group', 'full_path': 'stupid-conference', 'parent_id': None}, 'name_with_namespace': 'stupid-conference / stream-hud', 'http_url_to_repo': 'https://gitlab.com/stupid-conference/stream-hud.git', 'description': 'Худ для стримов саймона', 'tag_list': [], 'default_branch': None, 'created_at': '2018-10-19T16:52:15.981Z', '_id': ObjectId('5bca0be728bac7005ebd5759'), 'avatar_url': None, 'path_with_namespace': 'stupid-conference/stream-hud', 'last_activity_at': '2018-10-19T16:52:15.981Z', 'id': 8952285, 'star_count': 0, 'forks_count': 0, 'readme_url': None}
{'web_url': 'https://gitlab.com/cocoort/jokes', 'path': 'jokes', 'name': 'jokes', 'ssh_url_to_repo': 'git@gitlab.com:cocoort/jokes.git', 'namespace': {'id': 3840487, 'path

{'web_url': 'https://gitlab.com/picomv/picomv.gitlab.io', 'path': 'picomv.gitlab.io', 'name': 'picomv.gitlab.io', 'ssh_url_to_repo': 'git@gitlab.com:picomv/picomv.gitlab.io.git', 'namespace': {'id': 3805700, 'path': 'picomv', 'name': 'pico', 'kind': 'group', 'full_path': 'picomv', 'parent_id': None}, 'name_with_namespace': 'pico / picomv.gitlab.io', 'http_url_to_repo': 'https://gitlab.com/picomv/picomv.gitlab.io.git', 'description': "Generated Pico's WebApplication", 'tag_list': [], 'default_branch': None, 'created_at': '2018-10-19T15:31:57.379Z', '_id': ObjectId('5bca0bee28bac7005ebd57be'), 'avatar_url': None, 'path_with_namespace': 'picomv/picomv.gitlab.io', 'last_activity_at': '2018-10-19T15:31:57.379Z', 'id': 8950969, 'star_count': 0, 'forks_count': 0, 'readme_url': None}
{'web_url': 'https://gitlab.com/doniks/ci-again', 'path': 'ci-again', 'name': 'ci-again', 'ssh_url_to_repo': 'git@gitlab.com:doniks/ci-again.git', 'namespace': {'id': 3038855, 'path': 'doniks', 'name': 'doniks', '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

